In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
data_dir = "/content/drive/MyDrive/Akash AI project/processed_data"
train_dir = data_dir + "/train"
test_dir = data_dir + "/Test"
print(train_dir, test_dir)



In [ ]:
import os

print("Train classes:", os.listdir(train_dir))
print("Test classes:", os.listdir(test_dir))



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import pandas as pd

# ✅ Enable eager execution fully (fixes callback error)
tf.config.run_functions_eagerly(True)

# ✅ Dataset Path (Google Drive)
base_dir = "/content/drive/MyDrive/Akash AI project/processed_data"
train_dir = base_dir + "/train"
test_dir = base_dir + "/Test"

# ✅ Load data
img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

# ✅ Prefetch for speed
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

num_classes = len(train_ds.class_names)
print("✅ Classes:", train_ds.class_names)

# ✅ Hyperparameters to evaluate
optimizers = ["adam", "rmsprop"]
activations = ["relu", "sigmoid", "tanh", "elu"]

epochs = 10
results = []

# ✅ Model Builder
def build_model(activation):
    model = models.Sequential([
        layers.Rescaling(1./255),
        layers.Conv2D(32, 3, activation=activation, input_shape=img_size + (3,)),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, activation=activation),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, activation=activation),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation=activation),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation="softmax")
    ])
    return model

# ✅ Train each combination
for opt in optimizers:
    for act in activations:
        print(f"\n🚀 Training → Optimizer={opt} | Activation={act}")

        model = build_model(act)
        model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

        history = model.fit(train_ds, epochs=epochs, validation_data=test_ds, verbose=1)

        best_acc = max(history.history['val_accuracy'])
        results.append([opt, act, best_acc])

        print(f"✅ Best Test Accuracy: {best_acc:.4f}")

# ✅ Results as Table
df = pd.DataFrame(results, columns=["Optimizer", "Activation", "Val Accuracy"])
print("\n📌 Final Results:")
print(df)

# ✅ Bar Graph
plt.figure(figsize=(10, 5))
plt.bar(df["Optimizer"] + " + " + df["Activation"], df["Val Accuracy"])
plt.xticks(rotation=45, ha="right")
plt.title("Comparison of Model Accuracy (10 Epochs)")
plt.xlabel("Optimizer + Activation")
plt.ylabel("Validation Accuracy")
plt.ylim(0, 1)
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.show()


In [ ]:
# ✅ Mount Google Drive First
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow sklearn matplotlib

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, Xception, InceptionV3, EfficientNetV2B0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os

# ✅ GPU Check
print("GPU:", tf.config.list_physical_devices('GPU'))

# ✅ Correct Dataset Path
train_dir = "/content/drive/MyDrive/Akash AI project/processed_data/train"
test_dir = "/content/drive/MyDrive/Akash AI project/processed_data/Test"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 50

# ✅ Data Loading
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

num_classes = len(train_gen.class_indices)
results = []

# ✅ Training Function
def train_model(base_model, model_name, optimizer_name, activation_func):

    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(base_model.input, output)

    # ✅ Choose optimizer
    optimizer = tf.keras.optimizers.Adam() if optimizer_name == "Adam" else tf.keras.optimizers.RMSprop()

    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    print(f"\n🔹 Training {model_name} | OPT: {optimizer_name} | ACT: {activation_func}")

    es = EarlyStopping(patience=5, restore_best_weights=True)

    history = model.fit(
        train_gen,
        validation_data=test_gen,
        epochs=EPOCHS,
        callbacks=[es],
        verbose=1
    )

    # ✅ Evaluation
    preds = model.predict(test_gen)
    preds_cls = np.argmax(preds, axis=1)
    true = test_gen.classes
    acc = accuracy_score(true, preds_cls)

    print(f"✅ Accuracy = {acc:.4f}")

    # ✅ Save result to table
    results.append([model_name, optimizer_name, activation_func, acc])


# ✅ Models, Optimizers & Activations Combinations
model_list = [
    (ResNet50(include_top=False, input_shape=(224,224,3), weights="imagenet"), "ResNet50"),

]

optimizers = ["Adam", "RMSprop"]
activations = ["relu", "elu", "sigmoid", "tanh"]

# ✅ 32 Runs Total
for base_model, name in model_list:
    for opt in optimizers:
        for act in activations:
            train_model(base_model, name, opt, act)

# ✅ Final Accuracy Results Table
df_results = pd.DataFrame(results, columns=["Model", "Optimizer", "Activation", "Accuracy"])
df_results = df_results.sort_values(by="Accuracy", ascending=False)

print("\n\n🏆 Final Accuracy Ranking Table")
print(df_results)

In [ ]:
# ✅ Mount Google Drive First
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow sklearn matplotlib

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, Xception, InceptionV3, EfficientNetV2B0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os

# ✅ GPU Check
print("GPU:", tf.config.list_physical_devices('GPU'))

# ✅ Correct Dataset Path
train_dir = "/content/drive/MyDrive/Akash AI project/processed_data/train"
test_dir = "/content/drive/MyDrive/Akash AI project/processed_data/Test"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 50

# ✅ Data Loading
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

num_classes = len(train_gen.class_indices)
results = []

# ✅ Training Function
def train_model(base_model, model_name, optimizer_name, activation_func):

    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(base_model.input, output)

    # ✅ Choose optimizer
    optimizer = tf.keras.optimizers.Adam() if optimizer_name == "Adam" else tf.keras.optimizers.RMSprop()

    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    print(f"\n🔹 Training {model_name} | OPT: {optimizer_name} | ACT: {activation_func}")

    es = EarlyStopping(patience=5, restore_best_weights=True)

    history = model.fit(
        train_gen,
        validation_data=test_gen,
        epochs=EPOCHS,
        callbacks=[es],
        verbose=1
    )

    # ✅ Evaluation
    preds = model.predict(test_gen)
    preds_cls = np.argmax(preds, axis=1)
    true = test_gen.classes
    acc = accuracy_score(true, preds_cls)

    print(f"✅ Accuracy = {acc:.4f}")

    # ✅ Save result to table
    results.append([model_name, optimizer_name, activation_func, acc])


# ✅ Models, Optimizers & Activations Combinations
model_list = [

    (Xception(include_top=False, input_shape=(224,224,3), weights="imagenet"), "Xception"),
    (InceptionV3(include_top=False, input_shape=(224,224,3), weights="imagenet"), "InceptionV3")
]

optimizers = ["Adam", "RMSprop"]
activations = ["relu", "elu", "sigmoid", "tanh"]

# ✅ 32 Runs Total
for base_model, name in model_list:
    for opt in optimizers:
        for act in activations:
            train_model(base_model, name, opt, act)

# ✅ Final Accuracy Results Table
df_results = pd.DataFrame(results, columns=["Model", "Optimizer", "Activation", "Accuracy"])
df_results = df_results.sort_values(by="Accuracy", ascending=False)

print("\n\n🏆 Final Accuracy Ranking Table")
print(df_results)

In [ ]:
# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Install Dependencies
!pip install tensorflow sklearn matplotlib

# ✅ Imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2, Regularizer
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os

# ✅ GPU Check
print("GPU:", tf.config.list_physical_devices('GPU'))

# ✅ Dataset Paths
train_dir = "/content/drive/MyDrive/Akash AI project/processed_data/train"
test_dir  = "/content/drive/MyDrive/Akash AI project/processed_data/Test"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 10  # Short training for underfitting/overfitting visualization

# ✅ Data Loading
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

num_classes = len(train_gen.class_indices)
results = []

# ✅ Custom L3 Regularizer (Keras-compatible)
class L3Regularizer(Regularizer):
    def __init__(self, l3=0.001):
        self.l3 = l3
    def __call__(self, x):
        return self.l3 * K.sum(K.pow(K.abs(x), 3))
    def get_config(self):
        return {'l3': float(self.l3)}

# ✅ Training Function
def train_model_with_regularization(base_model, model_name, reg_type="L2", reg_value=0.001, optimizer_name="Adam"):
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)

    # Select regularizer
    if reg_type == "L2":
        regularizer = l2(reg_value)
    elif reg_type == "L3":
        regularizer = L3Regularizer(reg_value)
    else:
        regularizer = None

    x = Dense(128, activation='relu', kernel_regularizer=regularizer)(x)
    x = Dropout(0.4)(x)
    output = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)

    optimizer = tf.keras.optimizers.Adam() if optimizer_name == "Adam" else tf.keras.optimizers.RMSprop()

    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    print(f"\n🔹 Training {model_name} | Regularizer: {reg_type}={reg_value} | Optimizer: {optimizer_name}")

    es = EarlyStopping(patience=3, restore_best_weights=True)

    start_time = time.time()
    history = model.fit(
        train_gen,
        validation_data=test_gen,
        epochs=EPOCHS,
        callbacks=[es],
        verbose=1
    )

    # ✅ Evaluate
    preds = model.predict(test_gen)
    preds_cls = np.argmax(preds, axis=1)
    true = test_gen.classes
    acc = accuracy_score(true, preds_cls)

    print(f"✅ Test Accuracy = {acc:.4f}")
    print(f"⏱️ Training Time: {time.time() - start_time:.2f}s")

    # ✅ Save Result
    results.append([model_name, reg_type, reg_value, optimizer_name, acc])

    # ✅ Plot Training vs Validation Accuracy
    plt.figure(figsize=(6,4))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} ({reg_type}) Accuracy Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    return model, history

# ✅ Run Experiments
models = [
    (Xception(include_top=False, input_shape=(224,224,3), weights="imagenet"), "Xception"),
    (InceptionV3(include_top=False, input_shape=(224,224,3), weights="imagenet"), "InceptionV3")
]

optimizers = ["Adam", "RMSprop"]

for base_model, name in models:
    for opt in optimizers:
        # L2 Regularization
        train_model_with_regularization(base_model, name, reg_type="L2", reg_value=0.001, optimizer_name=opt)
        # L3 Regularization
        train_model_with_regularization(base_model, name, reg_type="L3", reg_value=0.0001, optimizer_name=opt)

# ✅ Final Results Table
df_results = pd.DataFrame(results, columns=["Model", "Regularization", "Reg Value", "Optimizer", "Test Accuracy"])
df_results = df_results.sort_values(by="Test Accuracy", ascending=False)

print("\n\n🏆 Final Regularization Comparison Table")
print(df_results.to_markdown(index=False))
